In [23]:
import numpy as np
import pandas as pd
import glob
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [ ]:
# Define the folder where all your CSVs are stored
folder_path = "dataset/"  # Adjust this to your actual folder

# Load all CSV files with stripped column names
print("Loading data from CSV files...")
all_files = glob.glob(folder_path + "*.csv")
if not all_files:
    raise FileNotFoundError(f"No CSV files found in {folder_path}")

df_list = []
for file in all_files:
    try:
        print(f"Reading file: {file}")
        df = pd.read_csv(file, skipinitialspace=True)
        df.columns = df.columns.str.strip()  # Remove leading/trailing spaces
        df_list.append(df)
        print(f"Successfully loaded {len(df)} rows from {file}")
    except Exception as e:
        print(f"Error reading {file}: {str(e)}")
        continue

if not df_list:
    raise ValueError("No data was successfully loaded from any files")

# Concatenate all dataframes
print("\nConcatenating all dataframes...")
df = pd.concat(df_list, ignore_index=True)

In [25]:
# Calculate 70% of the total length for training
train_size = int(len(df) * 0.7)

# Generate random indices for training (70% of data)
train_indices = np.random.choice(np.arange(0, len(df)), train_size, replace=False)

# Get validation indices (remaining 30%)
val_indices = list(set(np.arange(0, len(df))) - set(train_indices))

In [26]:
df.shape

(108995, 79)

In [27]:
cols_to_del = ['DistanceCovered', 'FuelLevel', 'Gear', 'CurrentLapTime', 'Gear.1', 'Damage', 'RacePosition']

In [28]:
for col in cols_to_del:
    del df[col]

In [29]:
training_data = df.loc[train_indices].reset_index(drop=True)
validation_data = df.loc[val_indices].reset_index(drop=True)

In [43]:
training_data.columns

Index(['Angle', 'DistanceFromStart', 'LastLapTime', 'Opponent_1', 'Opponent_2',
       'Opponent_3', 'Opponent_4', 'Opponent_5', 'Opponent_6', 'Opponent_7',
       'Opponent_8', 'Opponent_9', 'Opponent_10', 'Opponent_11', 'Opponent_12',
       'Opponent_13', 'Opponent_14', 'Opponent_15', 'Opponent_16',
       'Opponent_17', 'Opponent_18', 'Opponent_19', 'Opponent_20',
       'Opponent_21', 'Opponent_22', 'Opponent_23', 'Opponent_24',
       'Opponent_25', 'Opponent_26', 'Opponent_27', 'Opponent_28',
       'Opponent_29', 'Opponent_30', 'Opponent_31', 'Opponent_32',
       'Opponent_33', 'Opponent_34', 'Opponent_35', 'Opponent_36', 'RPM',
       'SpeedX', 'SpeedY', 'SpeedZ', 'Track_1', 'Track_2', 'Track_3',
       'Track_4', 'Track_5', 'Track_6', 'Track_7', 'Track_8', 'Track_9',
       'Track_10', 'Track_11', 'Track_12', 'Track_13', 'Track_14', 'Track_15',
       'Track_16', 'Track_17', 'Track_18', 'Track_19', 'TrackPosition',
       'WheelSpinVelocity_1', 'WheelSpinVelocity_2', 'WheelS

In [31]:
# For training data
X_train = training_data.loc[:, 'Angle':'Z']  # Input features from Angle to Z
Y_train = training_data.loc[:, 'Acceleration':'Steering']  # Target variables from Acceleration to Steering

# For validation data
X_val = validation_data.loc[:, 'Angle':'Z']  # Input features from Angle to Z
Y_val = validation_data.loc[:, 'Acceleration':'Steering']  # Target variables from Acceleration to Steering

In [46]:
X_train

array([[ 0.36788233, -0.90237532, -1.17524787, ..., -0.42800734,
        -0.45792239,  0.66853406],
       [-0.07964823,  0.83692903, -0.03211006, ...,  0.91285012,
         0.91190227, -0.85527411],
       [ 0.5956033 ,  1.0169867 ,  1.90544639, ..., -1.1319364 ,
        -1.1076143 ,  0.90621282],
       ...,
       [-0.0050194 , -0.34772516,  0.51727948, ..., -1.44248732,
        -1.44087522,  1.17400334],
       [-0.00414516,  1.23394341,  0.66793759, ...,  0.15263044,
         0.15317748,  0.07150462],
       [-0.09387781,  1.11528908, -1.17524787, ...,  0.98291714,
         0.98536645, -1.09831174]])

In [32]:
print (X_train.shape)
print (Y_train.shape)
print (X_val.shape)
print (Y_val.shape)
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_val=scaler.fit_transform(X_val)
Y_train=Y_train.values
Y_val=Y_val.values
print (X_train.shape)
print (Y_train.shape)
print (X_val.shape)
print (Y_val.shape)

(76296, 68)
(76296, 4)
(32699, 68)
(32699, 4)
(76296, 68)
(76296, 4)
(32699, 68)
(32699, 4)


In [ ]:
scaler.scale_

In [ ]:
learning_rate=3e-3
output_dim=Y_train.shape[1]
model=Sequential()
model.add(Dense(1024
                ,input_shape=(X_train.shape[1],),activation='relu',kernel_initializer='he_normal'))
model.add(Dense(512,activation='relu',kernel_initializer='he_normal'))
model.add(Dense(256,activation='relu',kernel_initializer='he_normal'))
model.add(Dense(128,activation='relu',kernel_initializer='he_normal'))
model.add(Dense(64,activation='relu',kernel_initializer='he_normal'))
model.add(Dense(32,activation='relu',kernel_initializer='he_normal'))
model.add(Dense(16,activation='relu',kernel_initializer='he_normal'))
model.add(Dense(8,activation='relu',kernel_initializer='he_normal'))
model.add(Dense(4,kernel_initializer='he_normal'))
model.compile(optimizer=Adam(learning_rate),loss='mean_squared_error',metrics=['mae'])
model.fit(X_train,Y_train,validation_data=(X_val,Y_val),batch_size=256,epochs=500)

In [37]:
model.predict(X_val[0].reshape(1,-1),batch_size=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step


array([[ 0.99790287, -0.00716882,  0.6713108 ,  0.1310945 ]],
      dtype=float32)

In [38]:
model.save(r'./sample_model.h5')

In [41]:
model=load_model(r'./sample_model.h5')

In [42]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 1024)           │        70,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 4)              │            36 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 770,718 (2.94 MB)

 Trainable params: 770,716 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)